## *Working with Gradio*

In [ ]:
import gradio as gr

In [ ]:
def shout(text):
    print(f"Shout has been called with input {text}")
    return text.upper()

In [ ]:
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(inbrowser=True)

In [ ]:
# Adding a little more

message_input = gr.Textbox(label="Your message", info="Enter a message to be shouted", lines=7)
message_output = gr.Textbox(label="Response", lines=7)

view = gr.Interface(
    fn=shout,
    title="Shout",
    inputs=[message_input],
    outputs=[message_output],
    examples=["hello", "woah!"],
    flagging_mode="never"
)
view.launch(share=True, inbrowser=True, auth=("nithin", "0704"))

In [ ]:
# Now getting llm to answer

import os
from openai import OpenAI
from dotenv import load_dotenv

In [ ]:
load_dotenv(override=True)

GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
api_key = os.getenv('GEMINI_API_KEY')

gemini = OpenAI(base_url=GEMINI_BASE_URL, api_key=api_key)

In [ ]:
system_prompt = "You are a helpful assistant that provides friendly responses to user queries."

def message_gemini(prompt):
    messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": prompt}]
    response = gemini.chat.completions.create(model='gemini-2.5-flash-lite', messages=messages)
    return response.choices[0].message.content

In [ ]:
message_input = gr.Textbox(label="Your Message", info="Ask anything..", lines = 7)
message_output = gr.Textbox(label="Response", lines=8)
view = gr.Interface(
    fn=message_gemini,
    title='Gemini 2.5 Flash Lite',
    inputs=[message_input],
    outputs=[message_output],
    examples=["Hello gemini", "What is AI?"],
    flagging_mode="never"
)
view.launch(inbrowser=True)

In [ ]:
# getting response in Markdown instead of in Textbox

message_input = gr.Textbox(label="Your message", info="Ask Anything..", lines=7)
message_output = gr.Markdown(label="Response:")
view = gr.Interface(
    fn=message_gemini,
    title="Gemini 2.5 Flash Lite",
    inputs=[message_input],
    outputs=[message_output],
    examples=[
        "Explain the Transformer architecture to a layperson",
        "Explain the Transformer architecture to an aspiring AI engineer",
        ], 
    flagging_mode="never"
)
view.launch(inbrowser=True)

In [ ]:
# Now lets stream the response of gemini 2.5 flash lite

def stream_gemini(prompt):
    messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": prompt}]
    stream = gemini.chat.completions.create(
        model='gemini-2.5-flash-lite',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [ ]:
message_input = gr.Textbox(label="Your message", info="Ask Anything..", lines=7)
message_output = gr.Markdown(label="Response:")
view = gr.Interface(
    fn=stream_gemini,
    title="Gemini 2.5 Flash Lite",
    inputs=[message_input],
    outputs=[message_output],
    examples=[
        "Explain the Transformer architecture to a layperson",
        "Explain the Transformer architecture to an aspiring AI engineer",
        ], 
    flagging_mode="never"
)
view.launch(inbrowser=True)

In [ ]:
GROQ_BASE_URL = "https://api.groq.com/openai/v1"
groq_api_key = os.getenv('GROQ_API_KEY')


groq = OpenAI(base_url=GROQ_BASE_URL, api_key=groq_api_key)

In [ ]:
def message_groq_llama(prompt):
    messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": prompt}]
    response = groq.chat.completions.create(
        model='llama-3.1-8b-instant',
        messages=messages
    )
    return response.choices[0].message.content

In [ ]:
message_input = gr.Textbox(label="Your message", info="Ask Anything..", lines=7)
message_output = gr.Markdown(label="Response:")
view = gr.Interface(
    fn=message_groq_llama,
    title="GROQ LLAMA",
    inputs=[message_input],
    outputs=[message_output],
    examples=[
        "Explain the Transformer architecture to a layperson",
        "Explain the Transformer architecture to an aspiring AI engineer",
        ], 
    flagging_mode="never"
)
view.launch(inbrowser=True)

In [ ]:
def message_groq_openai(prompt):
    messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": prompt}]
    response = groq.chat.completions.create(
        model='openai/gpt-oss-20b',
        messages=messages
    )
    return response.choices[0].message.content

In [ ]:
message_input = gr.Textbox(label="Your message", info="Ask Anything..", lines=7)
message_output = gr.Markdown(label="Response:")
view = gr.Interface(
    fn=message_groq_openai,
    title="GROQ OPENAI",
    inputs=[message_input],
    outputs=[message_output],
    examples=[
        "Explain the Transformer architecture to a layperson",
        "Explain the Transformer architecture to an aspiring AI engineer",
        ], 
    flagging_mode="never"
)
view.launch(inbrowser=True)

In [ ]:
def select_model(prompt, model):
    if model == "GROQ LLAMA":
        result = message_groq_llama(prompt)
    elif model == "GROQ OPENAI":
        result = message_groq_openai(prompt)
    else:
        raise ValueError("Unknown Model")
    return result

In [ ]:
message_input = gr.Textbox(label="Your message", info="Ask anything..", lines=7)
model_selector = gr.Dropdown(["GROQ LLAMA", "GROQ OPENAI"], label="Select model", value="GROQ LLAMA")
message_output = gr.Markdown(label="Response:")

view = gr.Interface(
    fn=select_model,
    title="LLMs",
    inputs=[message_input, model_selector],
    outputs=[message_output],
    examples=[
            ["Explain the Transformer architecture to a layperson", "GROQ LLAMA"],
            ["Explain the Transformer architecture to an aspiring AI engineer", "GROQ OPENAI"]
        ], 
    flagging_mode="never"
)
view.launch(share=True, inbrowser=True, auth=("nithin", "0704"))